# Compute geology indices
**Please clean all outputs when committing all the changes in git.**

In [ ]:
%matplotlib inline  
import os, sys
import xarray as xr
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import scripts.utility as util

print("\nThe Python version: %s.%s.%s" % sys.version_info[:3])

-------------------------
## 1. Setup <a id='setup'></a>

In [ ]:
setup = util.load_yaml("./geology_index_setup.yaml")

catchment_name = setup["catchment_name"]  # Catchment case: CONUS_HUC12, camels, camelsx-v3
serial = setup["serial"] # True: dask distributed enabled
saveCSV = setup["saveCSV"] # True: save attributes for each HRU in csv
saveNetCDF = setup["saveNetCDF"] # True: save attributes for each HRU in netcdf
remap = setup["remap"] # True: remap meteorological time series to HRUs

# files and directories
src_dir = setup["src_dir"]
catch_gpkg = setup["catch_gpkg"]
LiMW2015_mapping_file = setup["LiMW2015_mapping_file"]
GLHYMPS_mapping_file = setup["GLHYMPS_mapping_file"]
LiMW2015_gpkg = setup['LiMW2015_gpkg']
GLHYMPS_gpkg = setup['GLHYMPS_gpkg']

# geology variable meta
LiMW2015_attr = setup['LiMW2015_attr']
GLHYMPS_attr = setup['GLHYMPS_attr']
GLiM_class = setup['GLiM_class']

# catchment meta
catch_attrs = setup["catch_attrs"]

print('-- Setup:')
print(f" Dask not enabled: {serial}")
print(f" catchment_name: {catchment_name}")
print(f" saveCSV: {saveCSV}")
print(f" saveNetCDF: {saveNetCDF}")
print(f" remap: {remap}")
print(f" geology data directory: {src_dir}")
print(f" catchment gpkg: {catch_gpkg}")
print(f" LiMW2015_mapping_file: {LiMW2015_mapping_file}")
print(f" GLHYMPS_mapping_file: {GLHYMPS_mapping_file}")
print(f" LiMW2015_gpkg: {LiMW2015_gpkg}")
print(f" GLHYMPS_gpkg: {GLHYMPS_gpkg}")

## Reading LiMW2015 and GLHYMPS geopackage data

In [ ]:
%%time
print(f'Reading LiMW2015 geology data')
gdf_LiMW2015 = util.read_shps([os.path.join(src_dir,LiMW2015_gpkg)],list(LiMW2015_attr.keys()))
ds_LiMW2015 = gdf_LiMW2015.rename(columns={'OBJECTID':'hru'}).set_index('hru').to_xarray()
ds_LiMW2015 = ds_LiMW2015.rename({'xx':'lithological_class'})
ds_LiMW2015['carbonate_rock'] = xr.DataArray(np.ones(len(ds_LiMW2015['hru']), dtype=int), dims=ds_LiMW2015.dims, coords=ds_LiMW2015.coords)
ds_LiMW2015['carbonate_rock'] = ds_LiMW2015['carbonate_rock'].where(ds_LiMW2015['lithological_class']=='sc', 0)
ds_LiMW2015

In [ ]:
%%time
print(f'Reading GLHYMPS geology data')
gdf_GLHYMPS = util.read_shps([os.path.join(src_dir, GLHYMPS_gpkg)],list(GLHYMPS_attr.keys()))
ds_GLHYMPS = gdf_GLHYMPS.rename(columns={'OBJECTID':'hru'}).set_index('hru').to_xarray()
ds_GLHYMPS

## Re-mapping
- 1st dominant class
- 2nd cominant class
- porosity
- Permeability

In [ ]:
%%time
dr_mask = xr.where(np.isnan(ds_LiMW2015['carbonate_rock']),0,1)
a = util.remap_mode(xr.open_dataset(LiMW2015_mapping_file), ds_LiMW2015, dr_mask, ['lithological_class'])
a['1st_dominant_lithological_class_fraction'] = a['1st_dominant_lithological_class_fraction'].where(a['1st_dominant_lithological_class']!='N/A', 0)
a['2nd_dominant_lithological_class_fraction'] = a['2nd_dominant_lithological_class_fraction'].where(a['2nd_dominant_lithological_class']!='N/A', 0)

In [ ]:
%%time
# carbonate rock fraction
a1 = util.remap_mean(xr.open_dataset(LiMW2015_mapping_file), ds_LiMW2015, dr_mask, ['carbonate_rock'])
a = xr.merge([a, a1['carbonate_rock'].rename('carbonate_rock_frac')])

In [ ]:
%%time
dr_mask = xr.where(np.isnan(ds_GLHYMPS['Porosity']),0,1)
a1 = util.remap_mean(xr.open_dataset(GLHYMPS_mapping_file), ds_GLHYMPS, dr_mask, ['Porosity', 'Permeability_permafrost'])
a = xr.merge([a, a1])
a

## Dataset to Dataframe

In [ ]:
df = a.to_dataframe()

## Save in csv or netcdf

In [ ]:
if saveCSV:
    df.to_csv(os.path.join('output',f'{catchment_name}_geol.csv'), float_format='%g')
if saveNetCDF:
    a.to_netcdf(os.path.join('output',f'{catchment_name}_geol.nc'))

## Plotting

In [ ]:
%%time
gdf_camels = util.read_shps([catch_gpkg], [catch_attrs[catchment_name]['id']])

In [ ]:
gdf_camels = gdf_camels.merge(a[['2nd_dominant_lithological_class',
                                 '2nd_dominant_lithological_class_fraction',
                                 '1st_dominant_lithological_class', 
                                 '1st_dominant_lithological_class_fraction',
                                 'carbonate_rock_frac',
                                 'Porosity', 
                                 'Permeability_permafrost']].to_dataframe(),left_on=catch_attrs[catchment_name]['id'], right_index=True)

In [ ]:
var_name = '1st_dominant_lithological_class'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                legend=True, legend_kwds={'fontsize':'xx-small','loc':'best'},
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = '1st_dominant_lithological_class_fraction'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = 'carbonate_rock_frac'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = 'Porosity'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(0, 0.25),
                legend=True
               );
ax.set_title(var_name);
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)

In [ ]:
var_name = 'Permeability_permafrost'
fig, ax = plt.subplots(figsize=(6, 3), dpi=150)
gdf_camels.plot(ax=ax, column=var_name, cmap='turbo',
                norm=colors.Normalize(-20, -10),
                legend=True
               );
ax.set_title(f'{var_name} log90');
fig.tight_layout()
plt.savefig(f'./figures/{catchment_name}_geol_{var_name}.png', dpi=300)